# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [39]:
stocks= pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [6]:
symbol='AAPL'
# Since we are using free data we will use the sandbox URL which would return random data becasue it is not paid.
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'ALTA  KN/MTEE)A LBSSRGEDSCOQ(ALNG', 'calculationPrice': 'close', 'open': 152.39, 'openTime': 1647901431475, 'openSource': 'acfoiifl', 'close': 147.29, 'closeTime': 1678163791399, 'closeSource': 'ciiflfao', 'high': 152.39, 'highTime': 1631720866386, 'highSource': 'e1 led dcy5tiepeamru ni', 'low': 147.29, 'lowTime': 1696800632797, 'lowSource': 'ne  ieemtep1di rd5aycul', 'latestPrice': 147.61, 'latestSource': 'Close', 'latestTime': 'August 6, 2021', 'latestUpdate': 1695265941902, 'latestVolume': 55495365, 'iexRealtimePrice': 149.42, 'iexRealtimeSize': 171, 'iexLastUpdated': 1662551586560, 'delayedPrice': 151.09, 'delayedPriceTime': 1651633172568, 'oddLotDelayedPrice': 148.252, 'oddLotDelayedPriceTime': 1686705500462, 'extendedPrice': 150.59, 'extendedChange': -0.18, 'extendedChangePercent': -0.00124, 'extendedPriceTime': 1701315039447, 'previousClose': 149.08, 'previousVolume': 48356057, 'change': -0.7, 'changePercent': -0

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']
price,market_cap

(147.61, 2498635350987)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker','Stock_Price','Market_cap','Number_of_shares_to_buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock_Price,Market_cap,Number_of_shares_to_buy


In [7]:
final_dataframe.append(
pd.Series(
[
    symbol,
    price,
    market_cap,
    'N/A'
],
    index = my_columns
),
    ignore_index= True
)

,Ticker,Stock_Price,Market_cap,Number_of_shares_to_buy
0,AAPL,150.95,2500606731763,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    data = requests.get(api_url).json()
    price = data['latestPrice']
    market_cap = data['marketCap']
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
        index = my_columns
    ),
        ignore_index= True
    )
final_dataframe

In [ ]:
final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


In [41]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
# After dividing the stocks into baskets of hundred we join the batches using ','
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

symbol_strings


['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [33]:
my_columns = ['Ticker','Stock_Price','Market_cap','Number_of_shares_to_buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

# We run a loop through the batches
for symbol_string in symbol_strings:
#     We make the API call for a batch
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,news,chart&range=1m&last=10&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     We run a for loop for each stock inside the active batch selected and split according to the ','
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
            ],
            index = my_columns
        ),
            ignore_index= True
        )
final_dataframe
# symbol_strings

,Ticker,Stock_Price,Market_cap,Number_of_shares_to_buy
0,A,160.90,48905948091,N/A
1,AAL,21.96,14143194349,N/A
2,AAP,205.81,13758785523,N/A
3,AAPL,147.03,2500879746567,N/A
4,ABBV,114.90,212208348016,N/A
...,...,...,...,...
500,YUM,137.03,40729457181,N/A
501,ZBH,155.00,31767212577,N/A
502,ZBRA,589.04,31071447996,N/A
503,ZION,57.53,9331563156,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [45]:
portfolio_size = input('Enter the value of the Portfolio')
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Thats not a number')
    portfolio_size = input('Enter the value of the Portfolio again')
    val = float(portfolio_size)
    

Enter the value of the Portfolio10000000
10000000.0


In [49]:
position_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number_of_shares_to_buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock_Price'])

final_dataframe


,Ticker,Stock_Price,Market_cap,Number_of_shares_to_buy
0,A,160.90,48905948091,123
1,AAL,21.96,14143194349,901
2,AAP,205.81,13758785523,96
3,AAPL,147.03,2500879746567,134
4,ABBV,114.90,212208348016,172
...,...,...,...,...
500,YUM,137.03,40729457181,144
501,ZBH,155.00,31767212577,127
502,ZBRA,589.04,31071447996,33
503,ZION,57.53,9331563156,344


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [50]:
writer = pd.ExcelWriter('Recommended Trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [59]:
# These will format the color of the excel
background_color = '#0a0a23'
Font_color = '#ffffff'

string_format = writer.book.add_format(

{
    'font_color': Font_color,
    'bg_color': background_color,
    'border':1
})

dollar_format = writer.book.add_format(

{
    'num_format' : '$0.00',
    'font_color': Font_color,
    'bg_color': background_color,
    'border':1
})

integer_format = writer.book.add_format(

{
    'num_format' : '0',
    'font_color': Font_color,
    'bg_color': background_color,
    'border':1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [62]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', integer_format],
    'D':['Number_of_shares_to_buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18, column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [63]:
writer.save()